# 0) Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %matplotlib inline

In [3]:
import os
import numpy as np
import pandas as pd
from time import time

import matplotlib.pyplot as plt

In [4]:
from itertools import cycle
from random import shuffle
from plotly import graph_objects as go
import plotly.express as px
from plotly.validators.scatter.marker import SymbolValidator
from PIL import ImageColor
import functools

COLORS = px.colors.qualitative.Plotly

LINE_STYLES = ["solid", "dot", "dash", "longdash", "dashdot", "longdashdot"]

SYMBOLS = [
    "circle",
    "square",
    "star",
    "x",
    "triangle-up",
    "pentagon",
    "cross",
]

# 1) Personal tools and solvers

In [5]:
from utils import is_symmetric, is_pos_def, is_commutative, res_norm_pr, rel_err_pr
from plots import plot_conv, plot_eigenvals
from results import ResultPR

from gen_data import gen_artifical_pr
from adi_peaceman_rachford import adi_pr, sap_adi_pr
from shift_parameters import shifts_pr, shifts_w

from save_load import save, save_artificial_pr, load_artificial_pr

### Parameters to select

In [6]:
N = 1000 # dimension
overwrite = True # overwrite matrices
n_iter_moving_shifts = 10 # number iterations PR and W shifts
n_iter_cst_shifts = 25 # number iterations cst shifts
n_iter = 50 # number iterations SPR

is_plot_spectrum = False # plot spectrum of matrices A and B


# Plots title
title = f"Peaceman-Rachford problem N={N}"

# 2) Generate input matrices and vector

## Load or generate data

In [7]:
filename = "artificial_pr_" + str(N) + ".npz"
folder = os.path.join(os.getcwd(), "datasets/pr/")
path = os.path.join(folder, filename)

if os.path.exists(path) and not overwrite:
    H, V, b, u_true = load_artificial_pr(N)
    print("Matrices loaded from\n", path)
else:
    np.random.seed(0)
    H, V, b, u_true = gen_artifical_pr(N)
    save_artificial_pr(H, V, b, u_true)
    print("Matrices saved in\n", path)
    
m = H.shape[0]
print("Shape of H :", m)

Matrices saved in
 /home/claire/nidham/thomas_jefferson_fund/sketch_proj_ADI/code/sketched_adi/datasets/pr/artificial_pr_1000.npz
Shape of H : 1000


In [8]:
# Check that u_true is a solution
if u_true.size == 0:
    print("Solution not provided")
else:
    if res_norm_pr(u_true, H, V, b) < 1e-3:
        print("The problem is well posed")
#         u_direct = np.linalg.solve(H + V, b) # computed solution
#         print(res_norm_pr(u_direct, H, V, b))
#         print(np.linalg.norm(u_direct - u_true)) # u_true: ground truth solution
    else:
        print("The problem does not have a solution")
        u_true = np.array([])

The problem is well posed


In [9]:
# Estimate eigenvalue interval
t0 = time()
eig_val_H = np.linalg.eigvalsh(H)
eig_val_V = np.linalg.eigvalsh(V)

all_eig_val = np.concatenate((eig_val_H, eig_val_V))
alpha = np.min(all_eig_val)
beta = np.max(all_eig_val)
t_eigvalsh = time() - t0

print(
    f"Spectrum of H: [{np.min(eig_val_H):.5f}, {np.max(eig_val_H):.5f}]"
    f" => condition number (H) = {np.linalg.cond(H):.0f}"
)
print(
    f"Spectrum of V: [{np.min(eig_val_V):.5f}, {np.max(eig_val_V):.5f}]"
    f" => condition number (V) = {np.linalg.cond(V):.0f}"
)

print(f"Eigenvalue interval [alpha, beta] = [{alpha}, {beta}]\n")
print(f"Time numpy eigvalsh: {t_eigvalsh:.2} s\n")

print(f"Condition number (H + V) = {np.linalg.cond(H + V):.0f}")

Spectrum of H: [0.00777, 1.00100] => condition number (H) = 129
Spectrum of V: [0.00155, 1.00150] => condition number (V) = 648
Eigenvalue interval [alpha, beta] = [0.0015458562066422107, 1.0014999999999947]

Time numpy eigvalsh: 0.12 s

Condition number (H + V) = 215


## Remark
H + V = H - (- V), sp(H) and sp(-V) must be separated enough
a = min eig (H), b = max eig (H)
c = min eig (-V), d = max eig (-V)

gam = (c - a )*( d - b )/( c - b )/( d - a ); % Cross - ratio of a ,b ,c , d
=> Measures the "difficulty" of the problem 

AX - XB = F when the eigenvalues of A ( B ) are in [a , b ] and
% the eigenvalues of B ( A ) are in [c , d ]

In [10]:
if is_plot_spectrum:
    # Plot eigenvalues decrease
    plot_eigenvals(eig_val_H, eig_val_V, "pr")

# 3) Finding the correct number of iterations on ADI with Peaceman-Rachford shifts

In [11]:
results_iter_exp = []

In [12]:
for n_iter_tmp in [5, 10, 15, 20]:
#     print(f"Problem of size: {N} | number of iterations: {n_iter_tmp}\n")
    
    # Compute PR shifts
    t0 = time()
    p_pr, q_pr = shifts_pr(alpha, beta, n_iter_tmp)
    t_init = time() - t0
    
    u_adi_pr, t_adi_pr, iter_adi_pr, epoch_adi_pr = adi_pr(
        H, V, b, 
        n_iter=n_iter_tmp, 
        p=p_pr, q=q_pr,
        store_every=1,
        verbose=False,
    )

    # Taking initialization into account
    # t_eigvalsh : time to compute spectrum bounds
    t_adi_pr += t_eigvalsh + t_init

    result_adi_pr = ResultPR(
        str(n_iter_tmp),
        u_adi_pr, t_adi_pr, iter_adi_pr, epoch_adi_pr,
        u_true,
    )
    result_adi_pr.compute_errors(H, V, b)

    results_iter_exp.append(result_adi_pr)

#     print(f"\nRelative residual of last iterate: {result_adi_pr.rel_res[-1]:.2e}")
#     if not result_adi_pr.rel_err.size == 0:
#         print(f"Relative error of last iterate: {result_adi_pr.rel_err[-1]:.2e}")
#     print("\n")

## Relative distance to optimum plot 

In [15]:
fig_number_iter_rel_err = go.Figure()

fig_number_iter_rel_err.update_layout(
    template="plotly_white", 
    font=dict(size=20,),
)

for result in results_iter_exp:
    x_array = result.iterations

    fig_number_iter_rel_err.add_trace(
        go.Scatter(
        x=x_array,
        y=result.rel_err,
        name=result.algo_name,
        mode="lines+markers",
#         line=dict(color=COLORS[0], dash=LINE_STYLES[0]),
        marker=dict(symbol=SYMBOLS[0], size=10),
    )
)

y_dist = "$\|u^k - u^*\|_2 \ / \ \|u^0 - u^*\|_2$"

fig_number_iter_rel_err.update_layout(
    margin={"l": 20, "r": 20, "t": 20, "b": 20},
    xaxis_title="iteration",
#     yaxis_title="relative residual norm",
    yaxis_title=y_dist,
    yaxis_type="log",
    showlegend=False,
#     legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
)

save_path = os.path.join(os.getcwd(), "thesis_plots")
full_path = os.path.join(save_path, f"finding_number_iterations_moving_shifts_pr_rel_err_1000.pdf")
fig_number_iter_rel_err.write_image(full_path)

## Relative residual plot 

In [16]:
fig_number_iter_rel_res = go.Figure()

fig_number_iter_rel_res.update_layout(
    template="plotly_white", 
    font=dict(size=20,),
)

for result in results_iter_exp:
    x_array = result.iterations

    fig_number_iter_rel_res.add_trace(
        go.Scatter(
        x=x_array,
        y=result.rel_res,
        name=result.algo_name,
        mode="lines+markers",
#         line=dict(color=COLORS[0], dash=LINE_STYLES[0]),
        marker=dict(symbol=SYMBOLS[0], size=10),
    )
)

y_res = "$\|(H+V)u^k - b \|_2 \ / \ \|(H+V)u^0 - b \|_2$"

fig_number_iter_rel_res.update_layout(
    margin={"l": 20, "r": 20, "t": 20, "b": 20},
    xaxis_title="iteration",
#     yaxis_title="relative residual norm",
    yaxis_title=y_res,
    yaxis_type="log",
    showlegend=False,
#     legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
)

save_path = os.path.join(os.getcwd(), "thesis_plots")
full_path = os.path.join(save_path, f"finding_number_iterations_moving_shifts_pr_rel_res_1000.pdf")
fig_number_iter_rel_res.write_image(full_path)